In [8]:
#model training 是使用 jester資料5種動作


# from  pynput import mouse, keyboard
# from pynput.keyboard import Key
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import cv2
import time
from keras.models import Model,load_model
# import keras
# import switch_1
#model1_name = ""
model_path = "model\\3DCNN+3LSTM_128_4.h5"
model = load_model(model_path)
#字型
font = cv2.FONT_HERSHEY_SIMPLEX
img_rows,img_cols=128, 128
cap = cv2.VideoCapture(0)
cv2.namedWindow('Original', cv2.WINDOW_NORMAL)

# set rt size as 640x480
ret = cap.set(3,640)
ret = cap.set(4,480)
framecount = 0
fps = ""
start = time.time()
frames = []
num=[3]
max=[1]
real_index = 5
instruction = 'No Gesture'
pre =0

#load 種類CSV檔
# class_file = 'class_jester_6_300.csv'
# with open(class_file) as f:
#     classes = f.readlines()
# classes = [c.strip() for c in classes]
classes = ['jap','hook','uppercut','none']
num_classes = 4

while(1):
    #fpss = cap.get(5)
    #print ("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}".format(fpss))
    ret, frame = cap.read()
    frame = cv2.flip(frame, 3)
    frame = cv2.resize(frame, (640,480))
    
    framecount = framecount + 1
    end  = time.time()
    timediff = (end - start)
    if( timediff >= 1):
        #timediff = end - start
        fps = 'FPS:%s' %(framecount)
        start = time.time()
        framecount = 0

    #(影像, 文字, 座標, 字型, 大小, 顏色, 線條寬度, 線條種類)
    cv2.putText(frame,fps,(10,20), font, 0.7,(0,255,0),2,1)
    X_tr=[]
         
    image=cv2.resize(frame,(img_rows,img_cols),interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    frames.append(gray)
    input=np.array(frames)
    
    if input.shape[0]==24:
        frames = []
        # print(input.shape)
        X_tr.append(input)
        X_train= np.array(X_tr)
        # print(X_train.shape)
        train_set = np.zeros((1, 24, img_cols,img_rows,3))
        train_set[0][:][:][:][:]=X_train[0,:,:,:,:]
        train_set = train_set.astype('float32')
        train_set -= 107.70251
        train_set /= 147.29749
        result = model.predict(train_set)
        num = np.argmax(result,axis =1)
        max = np.max(result,axis = 1)
#         print(num)
#         action = str(classes[int(num[0])])
#         print(max)
#         result = str(classes[int(num[0])])
#         num = 'gesture:%s' %(classes[int(num[0])])
        
#         print(num)
        input=[]
        
    cv2.putText(frame, 'confidence: ' + str(max[0]), (10, 100), font, 0.7, (0, 255, 0), 2, 1)
    cv2.putText(frame, 'gesture class: ' + str(classes[int(num[0])]), (10, 50), font, 0.7, (0, 255, 0), 2, 1)

    cv2.imshow('Original',frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()
